In [1]:
from IPython.display import Image

- https://github.com/deepseek-ai/DeepSeek-Math

## basics

- Data Collection and Decontamination: **revelant** data at scale
    - we create the DeepSeekMath Corpus, a large-scale high-quality pre-training corpus comprising 120B math tokens.
    - iterative pipeline;
    - seed corpus
        - a small but high-quality collection of math-related dataset
        - OpenWebMath
    - train a FastText model: classifier
        - revelant or irrelevant
    - Discover math-related domains
        - After the first iteration of data collection, numerous mathematical web pages remain uncollected,  ...
        - 扩召回；
- GRPO

In [2]:
Image(url='./imgs/ds-math-data.png', width=400)

## GRPO

- 不需要 value model，释放内存和计算；